In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

24/03/31 17:30:12 WARN Utils: Your hostname, Mds-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en0)
24/03/31 17:30:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 17:30:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from flask import Flask, jsonify
app = Flask(__name__)

In [4]:
covidDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("data.csv")

In [75]:
covidDF.show()

+-----------+---------+-------+---------+------------+--------------+
|    Country|    Cases| Deaths|Recovered|Active_Cases|Critical_Cases|
+-----------+---------+-------+---------+------------+--------------+
|      India| 45033893| 533540|        0|    44500353|             0|
|        USA|111752653|1218763|109682509|      851381|          1111|
|      China|   503302|   5272|   379053|      118977|             0|
|     Russia| 24082671| 402612| 23500207|      179852|             0|
|     Brazil| 38694221| 710966| 36249161|     1734094|             0|
|     France| 40138560| 167642| 39970918|           0|             0|
|    Germany| 38826304| 182891| 38240600|      402813|             0|
|         UK| 24910387| 232112| 24675212|        3063|             0|
|      Italy| 26722143| 196446| 26356336|      169361|            27|
|      Spain| 13914811| 121760| 13762417|       30634|           231|
|  Argentina| 10094643| 130733|  9963910|           0|             0|
|  Australia| 118435

In [6]:
# Most affected country among all the countries ( total death/total covid cases).

most_affected_country = covidDF.select("Country", (covidDF.Deaths / covidDF.Cases).alias("Death Rate")).orderBy("Death Rate", ascending=False).limit(1)
most_affected_country.show()



+-------+-------------------+
|Country|         Death Rate|
+-------+-------------------+
| Mexico|0.04348517534317676|
+-------+-------------------+



In [7]:
# Least affected country among all the countries ( total death/total covid cases).
least_affected_country = covidDF.select("Country", (covidDF.Deaths / covidDF.Cases).alias("Death Rate")).orderBy("Death Rate", descending=False).limit(1)

least_affected_country.show()


+--------+--------------------+
| Country|          Death Rate|
+--------+--------------------+
|S. Korea|0.001039399861268...|
+--------+--------------------+



In [59]:
# Country with highest covid cases.
highest_covid_cases = covidDF.select("Country", (covidDF.Deaths).alias("Total Deaths")).orderBy("Total Deaths", ascending=False).limit(1)
highest_covid_cases.show()


+-------+------------+
|Country|Total Deaths|
+-------+------------+
|    USA|     1218763|
+-------+------------+



In [60]:
minimum_covid_cases = covidDF.select("Country", (covidDF.Deaths).alias("Total Deaths")).orderBy("Total Deaths", ascending=True).limit(1)
minimum_covid_cases.show()

+-------+------------+
|Country|Total Deaths|
+-------+------------+
|  Ghana|        1462|
+-------+------------+



In [12]:
# Total cases.
total_cases = covidDF.select(covidDF.agg({"Cases": "sum"})).alias("Total Cases")
total_cases.show()

total_cases = covidDF.agg({"Cases": "sum"}).collect()[0][0]
print(total_cases)

PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got DataFrame.

In [62]:
# Country that handled the covid most efficiently( total recovery/ total covid cases).
most_efficient_country = covidDF.select("Country", (covidDF.Recovered / covidDF.Cases).alias("Recovery Rate")).orderBy("Recovery Rate", ascending=False).limit(1)
most_efficient_country.show()

+--------+------------------+
| Country|     Recovery Rate|
+--------+------------------+
|S. Korea|0.9989606001387312|
+--------+------------------+



In [63]:
# Country that handled the covid least efficiently( total recovery/ total covid cases)

least_efficient_country = covidDF.select("Country", (covidDF.Recovered / covidDF.Cases).alias("Recovery Rate")).orderBy("Recovery Rate", ascending=True).limit(1)
least_efficient_country.show()

+-------+-------------+
|Country|Recovery Rate|
+-------+-------------+
|  India|          0.0|
+-------+-------------+



In [65]:
# Country least suffering from covid (least critical cases).
least_critical_case = covidDF.select("Country", (covidDF.Critical_Cases).alias("Critical Cases")).orderBy("Critical Cases", ascending=True).limit(1)
least_critical_case.show()


+-------+--------------+
|Country|Critical Cases|
+-------+--------------+
|  India|             0|
+-------+--------------+



In [66]:
# Country still suffering from covid (highest critical cases).

least_critical_cases = covidDF.select("Country", (covidDF.Critical_Cases).alias("Critical Cases")).orderBy("Critical Cases", ascending=False).limit(1)
least_critical_cases.show()

+-------+--------------+
|Country|Critical Cases|
+-------+--------------+
|    USA|          1111|
+-------+--------------+

